# EEG Comfort Score Prediction
 
# This notebook implements an improved approach to predicting comfort scores from EEG features.


## 1. Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.feature_selection import SelectKBest, f_regression, f_classif, mutual_info_classif, RFE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVR, SVC
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool, cv
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                            f1_score, roc_auc_score, confusion_matrix, 
                            roc_curve, precision_recall_curve, 
                            average_precision_score)


import mrmr
from mrmr import mrmr_classif, mrmr_regression

import warnings

# Set random seed for reproducibility
np.random.seed(42)

## 2. Load and Prepare Data

In [2]:
# Load the data
file_path = "features-Master.csv"
data = pd.read_csv(file_path)

# Shuffle the data
shuffled = data.sample(frac=1, random_state=42).reset_index(drop=True)
data_shuffled = shuffled.iloc[:, 4:]
labels_shuffled = shuffled["Comfort Score"]

print("Number of Comfort Score == 3:", (labels_shuffled == 3).sum())

# Create binary labels (1,2 = 0; 4,5 = 1; exclude 3 for clearer separation)
binary_labels = labels_shuffled.apply(lambda x: 0 if x <= 2 else (1 if x >=4 else np.nan))
binary_data = data_shuffled[~binary_labels.isna()]
binary_labels = binary_labels[~binary_labels.isna()] ###############################################

Number of Comfort Score == 3: 100


## 3. Data Splitting and Preprocessing

In [3]:
# For binary classification
X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(
    binary_data,
    binary_labels,
    test_size=0.2,
    stratify=binary_labels,
    random_state=42
)

## 4. Scale features
- Make sure these are used consistently in all future code

In [4]:
scaler = StandardScaler()
X_train_bin_scaled = pd.DataFrame(scaler.fit_transform(X_train_bin))
X_test_bin_scaled = pd.DataFrame(scaler.transform(X_test_bin))

## 5. Model Training and Evaluation

In [5]:
# Function to evaluate models
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name=""):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # For regression
    if isinstance(model, (RandomForestRegressor, GradientBoostingRegressor, 
                         SVR, XGBRegressor, CatBoostRegressor)):
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        print(f"{model_name} - RMSE: {rmse:.4f}, R²: {r2:.4f}")
        return r2
    
    # For classification
    else:
        accuracy = accuracy_score(y_test, y_pred)
        print(f"{model_name} - Accuracy: {accuracy:.4f}")
        
        # Plot confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(6, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title(f'Confusion Matrix - {model_name}')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.show()
        
        return accuracy

### 5.1 Binary Classification Models

In [6]:
# Initialize classification models
class_models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42),
    "SVM": SVC(random_state=42)
}

# Evaluate each model
print("Binary Classification Performance:")
#for name, model in class_models.items():
#    evaluate_model(model, X_train_bin_scaled, X_test_bin_scaled, 
#                  y_train_bin, y_test_bin, name)

Binary Classification Performance:


In [7]:
class MRMRTransformer:
    def __init__(self, k_features):
        self.k_features = k_features
        self.selected_features = None
        self.column_names = None
    
    def fit(self, X, y):
        # Convert to DataFrame if not already
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        
        # Reset indices to avoid alignment issues
        X = X.reset_index(drop=True)
        y = pd.Series(y).reset_index(drop=True)
        
        self.column_names = X.columns.tolist()
        try:
            self.selected_features = mrmr_regression(X, y, K=self.k_features)
        except:
            # Fallback to random features if MRMR fails
            self.selected_features = np.random.choice(X.columns, size=min(self.k_features, len(X.columns)), replace=False)
        return self
    
    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X, columns=self.column_names)
        return X[self.selected_features]

# Binary Classifcation

In [ ]:
X = X_train_bin_scaled.copy()
y = y_train_bin.copy()

def binary_classification_objective(trial):
    # 1. Imputation
    impute_method = trial.suggest_categorical('imputation', ['mean', 'median', 'knn', 'iterative'])
    if impute_method == 'mean':
        imputer = SimpleImputer(strategy='mean')
    elif impute_method == 'median':
        imputer = SimpleImputer(strategy='median')
    elif impute_method == 'knn':
        imputer = KNNImputer(n_neighbors=trial.suggest_int('knn_neighbors', 3, 15))
    else:
        imputer = IterativeImputer(
            max_iter=trial.suggest_int('iterative_max_iter', 10, 50),
            random_state=42,
            tol=0.01
        )

    # 2. Feature selection
    fs_method = trial.suggest_categorical('feature_selection', ['ANOVA', 'MutualInfo', 'RFE', 'None'])
    selector = 'passthrough'  # Default if no feature selection
    
    if fs_method != 'None':
        k_features = trial.suggest_int('k_features', 10, min(50, X.shape[1]))
        
        if fs_method == 'ANOVA':
            selector = SelectKBest(f_classif, k=k_features)
        elif fs_method == 'MutualInfo':
            selector = SelectKBest(mutual_info_classif, k=k_features)
        elif fs_method == 'RFE':
            rfe_step = trial.suggest_float('rfe_step', 0.1, 1.0)
            estimator = RandomForestClassifier(
                n_estimators=trial.suggest_int('rfe_n_estimators', 50, 200),
                max_depth=trial.suggest_int('rfe_max_depth', 3, 10),
                random_state=42
            )
            selector = RFE(estimator, n_features_to_select=k_features, step=rfe_step)
    
    # 3. Model selection and hyperparameters
    model_name = trial.suggest_categorical('model', ['RandomForest', 'GradientBoosting', 'XGBoost', 'SVM', 'CatBoost'])
    
    if model_name == 'RandomForest':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 20),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
            'random_state': 42
        }
        model = RandomForestClassifier(**params)
        
    elif model_name == 'GradientBoosting':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'random_state': 42
        }
        model = GradientBoostingClassifier(**params)
        
    elif model_name == 'XGBoost':
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 5),
            'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
            'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
            'random_state': 42,
            'eval_metric': 'logloss',
            'use_label_encoder': False
        }
        model = XGBClassifier(**params)
        
    elif model_name == 'SVM':
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
        params = {
            'C': trial.suggest_float('C', 0.1, 100, log=True),
            'kernel': kernel,
            'gamma': trial.suggest_categorical('gamma', ['scale', 'auto']),
            'degree': trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3,
            'probability': True,
            'random_state': 42
        }
        model = SVC(**params)
        
    else:  # CatBoost
        params = {
            'iterations': trial.suggest_int('catboost_iterations', 100, 1000),
            'depth': trial.suggest_int('catboost_depth', 3, 10),
            'learning_rate': trial.suggest_float('catboost_learning_rate', 0.001, 0.3, log=True),
            'l2_leaf_reg': trial.suggest_float('catboost_l2_leaf_reg', 1e-8, 10.0, log=True),
            'border_count': trial.suggest_int('catboost_border_count', 32, 255),
            'random_strength': trial.suggest_float('catboost_random_strength', 1e-9, 10, log=True),
            'bagging_temperature': trial.suggest_float('catboost_bagging_temperature', 0.0, 1.0),
            'random_state': 42,
            'verbose': False
        }
        model = CatBoostClassifier(**params)

    # 4. Pipeline
    pipeline = Pipeline([
        ('imputer', imputer),
        ('feature_selection', selector),
        ('model', model)
    ])

    # 5. Cross-validation with proper scoring for classification
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    try:
        scores = cross_val_score(pipeline, X, y, cv=cv, scoring='roc_auc', n_jobs=-1)
        return np.mean(scores)
    except Exception as e:
        print(f"Error in trial: {e}")
        return -np.inf

# Run binary classification study
binary_study = optuna.create_study(direction='maximize')
binary_study.optimize(binary_classification_objective, n_trials=50, show_progress_bar=True)

# Print results
print("\nBinary Classification Optimization Results:")
print(f"Best ROC AUC: {binary_study.best_value:.4f}")
print("Best Parameters:")
for key, value in binary_study.best_params.items():
    print(f"  {key}: {value}")

[I 2025-08-13 14:23:51,057] A new study created in memory with name: no-name-3a3bf92d-b383-4f82-8e68-619a65602a52


  0%|          | 0/50 [00:00<?, ?it/s]

# Apply best imputation and feature selection to training data

In [ ]:
# Get best imputation method from study
best_impute_method = binary_study.best_params.get('imputation', 'mean')

# Apply the best imputation method
if best_impute_method == 'mean':
    imputer = SimpleImputer(strategy='mean')
elif best_impute_method == 'median':
    imputer = SimpleImputer(strategy='median')
elif best_impute_method == 'knn':
    imputer = KNNImputer(n_neighbors=study.best_params.get('knn_neighbors', 5))
elif best_impute_method == 'iterative':
    imputer = IterativeImputer(
        max_iter=study.best_params.get('iterative_max_iter', 50),
        random_state=42
    )

# Fit and transform the data
X_imputed = imputer.fit_transform(X)

# Convert back to DataFrame (if needed)
if isinstance(X, pd.DataFrame):
    X_imputed = pd.DataFrame(X_imputed, columns=X.columns)

# Now apply your feature selection code from before
best_fs_method = study.best_params.get('feature_selection', 'None')

if best_fs_method != 'None':
    k_features = study.best_params['k_features']
    
    if best_fs_method == 'ANOVA':
        selector = SelectKBest(f_regression, k=k_features)
    elif best_fs_method == 'MutualInfo':
        selector = SelectKBest(mutual_info_regression, k=k_features)
    elif best_fs_method == 'RFE':
        estimator = RandomForestRegressor(
            n_estimators=study.best_params.get('rfe_n_estimators', 100),
            max_depth=study.best_params.get('rfe_max_depth', 5),
            random_state=42
        )
        selector = RFE(
            estimator, 
            n_features_to_select=k_features,
            step=study.best_params.get('rfe_step', 1)
        )
    elif best_fs_method == 'MRMR':
        selector = MRMRTransformer(k_features=k_features)
    
    selector.fit(X_imputed, y)
    if hasattr(selector, 'get_support'):  # For SelectKBest/RFE
        selected_features = X.columns[selector.get_support()]
    else:  # For MRMRTransformer
        selected_features = selector.selected_features
    X_best = X_imputed[selected_features]
else:
    X_best = X_imputed
    selected_features = X.columns

# Apply best model and fit

In [ ]:
# Prepare test data with same transformations as training
# 1. Apply the same imputation
X_test_imputed = imputer.transform(X_test_bin_scaled)  # Use the already fitted imputer

# Convert back to DataFrame if needed
if isinstance(X_test_bin_scaled, pd.DataFrame):
    X_test_imputed = pd.DataFrame(X_test_imputed, columns=X_test_bin_scaled.columns)

# 2. Apply the same feature selection
if best_fs_method != 'None':
    if best_fs_method == 'MRMR':
        X_test_final = X_test_imputed[selected_features]
    else:
        X_test_final = selector.transform(X_test_imputed)  # Use the already fitted selector
        if isinstance(X_test_imputed, pd.DataFrame):
            X_test_final = pd.DataFrame(X_test_final, columns=selected_features)
else:
    X_test_final = X_test_imputed

# Make predictions
y_pred = best_model.predict(X_test_final)
y_pred_proba = best_model.predict_proba(X_test_final)[:, 1]  # Probabilities for class 1

# Calculate metrics
accuracy = accuracy_score(y_test_bin, y_pred)
precision = precision_score(y_test_bin, y_pred)
recall = recall_score(y_test_bin, y_pred)
f1 = f1_score(y_test_bin, y_pred)
roc_auc = roc_auc_score(y_test_bin, y_pred_proba)

print("\nFinal Model Evaluation on Test Set:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

# Confusion matrix
cm = confusion_matrix(y_test_bin, y_pred)
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'])
plt.title('Confusion Matrix - Best Binary Model')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# ROC Curve
fpr, tpr, _ = roc_curve(y_test_bin, y_pred_proba)
plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Precision-Recall Curve
precision_curve, recall_curve, _ = precision_recall_curve(y_test_bin, y_pred_proba)
average_precision = average_precision_score(y_test_bin, y_pred_proba)
plt.figure(figsize=(6,6))
plt.plot(recall_curve, precision_curve, label=f'Precision-Recall curve (AP = {average_precision:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.show()

# Evaluate with test data

In [ ]:
# Prepare test data with same transformations as training
# 1. Apply the same imputation
X_test_imputed = imputer.transform(X_test_bin_scaled)  # Use the already fitted imputer

# Convert back to DataFrame if needed
if isinstance(X_test_bin_scaled, pd.DataFrame):
    X_test_imputed = pd.DataFrame(X_test_imputed, columns=X_test_bin_scaled.columns)

# 2. Apply the same feature selection
if best_fs_method != 'None':
    if best_fs_method == 'MRMR':
        X_test_final = X_test_imputed[selected_features]
    else:
        X_test_final = selector.transform(X_test_imputed)  # Use the already fitted selector
        if isinstance(X_test, pd.DataFrame):
            X_test_final = pd.DataFrame(X_test_final, columns=selected_features)
else:
    X_test_final = X_test_imputed

# Make predictions
y_pred = best_model.predict(X_test_final)

# Calculate metrics
rmse = np.sqrt(mean_squared_error(y_test_bin, y_pred))
r2 = r2_score(y_test_bin, y_pred)

print("\nFinal Model Evaluation on Test Set:")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")

# Calculate accuracy within ±1 point
correct = np.sum(np.abs(y_test_bin - y_pred) <= 1)
accuracy = correct / len(y_test_bin)
print(f"Accuracy within ±1 point: {accuracy:.4f}")

# Confusion matrix
cm = confusion_matrix(y_test_bin, y_pred_bin)
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - Best Binary Model')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()